In [1]:
from coding import search_w

import numpy as np
from scipy import fftpack as FT

# Let's debug Cyclic Code a bit

## n = 7, s = 2, d = 10

In [29]:
# global variables:
n = 7
s = 2
#d = 10
d = 1

In [30]:
np.set_printoptions(precision=4,linewidth=200.0)
W, fake_W, W_perp, S = search_w(n, s)

In [31]:
def estimator_generator(n, s):
    estimator = np.zeros((n, s+1), dtype=complex)
    #z_gen_func = np.vectorize(lambda t: np.exp(-2*np.pi*t*1j/n))
    z_gen_func = np.vectorize(lambda t: np.exp(2*np.pi*t*1j/n))
    col1 = z_gen_func(np.arange(n))
    for i in range(s+1):
        estimator[:, i] = np.power(col1, i)
    return estimator

p_estimator = estimator_generator(n, s)
print(p_estimator)

[[ 1.0000+0.j      1.0000+0.j      1.0000+0.j    ]
 [ 1.0000+0.j      0.6235+0.7818j -0.2225+0.9749j]
 [ 1.0000+0.j     -0.2225+0.9749j -0.9010-0.4339j]
 [ 1.0000+0.j     -0.9010+0.4339j  0.6235-0.7818j]
 [ 1.0000+0.j     -0.9010-0.4339j  0.6235+0.7818j]
 [ 1.0000+0.j     -0.2225-0.9749j -0.9010+0.4339j]
 [ 1.0000+0.j      0.6235-0.7818j -0.2225-0.9749j]]


## simulate epsilon here (assume worker 0 1 fails):

In [32]:
simulated_epsilon = np.zeros((n, d), dtype=complex)
simulated_epsilon[0:2,:] = -100.0+0.0j
print(simulated_epsilon)

[[-100.+0.j]
 [-100.+0.j]
 [   0.+0.j]
 [   0.+0.j]
 [   0.+0.j]
 [   0.+0.j]
 [   0.+0.j]]


## obtain E_2 (simulated) here:

In [33]:
simulated_E2 = np.dot(W_perp, simulated_epsilon)
print(simulated_E2)

[[ -3.7430-16.3993j]
 [ -3.7430+16.3993j]
 [-29.3859+36.8488j]
 [-61.3621+29.5505j]]


## calculated a to get the full E matrix

In [34]:
import copy
poly_a = np.zeros(s+1, dtype=complex)
poly_a[-1] = 1+0j
fake_poly_a = copy.deepcopy(poly_a)
# calculate a over here:
E_2 = simulated_E2

def _cls_solver(A, b):
    return np.dot(np.dot(np.linalg.inv(np.dot(_array_getH(A), A)), _array_getH(A)),b)

def _array_getH(ndarray):
    # get conjugate transpose of a np.ndarray
    return ndarray.conj().T

_shape = E_2.shape
_s = _shape[0]/2
_d = _shape[1]
_X = np.take(E_2, np.array([range(-i-(_s+1), -i-2+1) for i in range(_s)]).reshape(-1), axis=0).reshape((_s, _s*_d), order='F')
# we use tmp_y as the start point to obtain the full E matrix
tmp_y = np.take(E_2, np.array([-i-1 for i in range(_s)]), axis=0)
_y = tmp_y.reshape(-1, order='F')
alpha = _cls_solver(np.transpose(_X), _y.reshape(_y.shape[0], 1))
poly_a[0:s] = alpha.reshape(-1)
fake_poly_a[0:s] = -alpha.reshape(-1)

# check matrix formula and calculated result:
print("Formulated X is :")
print(np.transpose(_X))
print
print("Formulated y is :")
print(_y.reshape(_y.shape[0], 1))
print
print("E_2 is: ")
print(E_2)
print
print("Polynomial a is: ")
print(poly_a)

Formulated X is :
[[ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]]

Formulated y is :
[[-61.3621+29.5505j]
 [-29.3859+36.8488j]]

E_2 is: 
[[ -3.7430-16.3993j]
 [ -3.7430+16.3993j]
 [-29.3859+36.8488j]
 [-61.3621+29.5505j]]

Polynomial a is: 
[-0.6235-0.7818j  1.6235+0.7818j  1.0000+0.j    ]


In [38]:
_X_v2 = np.take(E_2, np.array([range(-i-(_s+1), -i-2+1) for i in range(_s)]).reshape(-1), axis=0)

random_factor = np.random.normal(size=_d)
print("One dim of X_v2 {}".format(_X_v2[:, 0]))
print
aaa = np.dot(_X_v2, random_factor)
print("=======================================================")
print(_X_v2)
print(random_factor)
print("=======================================================")
_X_v2_4solve = aaa.reshape((_s, _s), order='F')
print
print("Combined X is {}".format(_X_v2_4solve))
tmp_y = np.take(E_2, np.array([-i-1 for i in range(_s)]), axis=0)

bbb = np.dot(tmp_y, random_factor)
alpha = _cls_solver(_X_v2_4solve, bbb.reshape(bbb.shape[0], 1))
print
print("solved a is: {}".format(alpha))
print("########################################################")

One dim of X_v2 [ -3.7430+16.3993j -29.3859+36.8488j  -3.7430-16.3993j  -3.7430+16.3993j]

[[ -3.7430+16.3993j]
 [-29.3859+36.8488j]
 [ -3.7430-16.3993j]
 [ -3.7430+16.3993j]]
[-0.4982]

Combined X is [[  1.8648 -8.1701j   1.8648 +8.1701j]
 [ 14.6401-18.3581j   1.8648 -8.1701j]]

solved a is: [[  1.4010e+00+0.8326j]
 [  2.2204e-16-2.5894j]]
########################################################


In [24]:
# function to obtain matrix E
def obtain_E(alpha, E_2, s):
    # obtain E_1 in shape of n-2s by d
    _processing_y = np.transpose(E_2)[:, -s:]

    for i in range(n-2*s):
        tmp = np.dot(_processing_y[:,-s:], alpha)
        print("Obtaining E[{}]".format(i))
        print(_processing_y[:,-s:])
        print
        print("Calculated Results:")
        print(tmp)
        print
        _processing_y = np.concatenate((_processing_y, tmp), axis=1)
        #print(_processing_y[0,-s:])
        #print('============================================================')
    #print(_processing_y[0, s:])
    #print("----------------------------------------------------------------")
    #exit()
    return np.transpose(_processing_y[:, s:])

# function to obtain epsilon
def obtain_epsilon(E):
    return FT.ifft(E, n=n, axis=0)

## d=10 has too many dims? Let's look just one dimension then

In [25]:
# if we look only one dimension: i.e. d=1
# check matrix formula and calculated result:
print("Formulated X (d=1) is :")
print(np.transpose(_X)[0:2, :])
print
print("Formulated y is (d=1) :")
print(_y.reshape(_y.shape[0], 1)[0:2])
print
print("E_2 is: ")
print(E_2[:, 0])
print
print("X * alpha, shoud equals to y")
print(np.dot(np.transpose(_X)[0:2, :],alpha))

Formulated X (d=1) is :
[[ -3.7430+16.3993j -29.3859+36.8488j]
 [ -3.7430-16.3993j  -3.7430+16.3993j]]

Formulated y is (d=1) :
[[-61.3621+29.5505j]
 [-29.3859+36.8488j]]

E_2 is: 
[ -3.7430-16.3993j  -3.7430+16.3993j -29.3859+36.8488j -61.3621+29.5505j]

X * alpha, shoud equals to y
[[-61.3621+29.5505j]
 [-29.3859+36.8488j]]


# Test New Method

In [19]:
q1 = np.exp(-2*np.pi*0*1j/n)
q2 = np.exp(-2*np.pi*1*1j/n)
q1_conj = np.conj(q1)
q2_conj = np.conj(q2)
print("q_1 is {}".format(q1))
print("q_2 is {}".format(q2))
print("q_1_conj is {}".format(q1_conj))
print("q_2_conj is {}".format(q2_conj))
#bbb = - (q1 + q2)
#ccc = q1 * q2
bbb = - (q1_conj + q2_conj)
ccc = q1_conj * q2_conj
print(bbb)
print(ccc)
print
real_poly=[ccc, bbb, 1]
print("The Correct polynomial should be: {}".format(real_poly))
print("The one we get is: {}".format(poly_a))
print
print(np.dot(p_estimator, real_poly))

q_1 is (1+0j)
q_2 is (0.623489801859-0.781831482468j)
q_1_conj is (1-0j)
q_2_conj is (0.623489801859+0.781831482468j)
(-1.62348980186-0.781831482468j)
(0.623489801859+0.781831482468j)

The Correct polynomial should be: [(0.62348980185873359+0.7818314824680298j), (-1.6234898018587336-0.7818314824680298j), 1]
The one we get is: [-0.6235-0.7818j  1.6235+0.7818j  1.0000+0.j    ]

[ 0.0000 +0.0000e+00j -1.2225 +5.8874e-01j -0.6784 +2.9725e+00j  2.3705 +2.9725e+00j  3.0489 +4.4409e-16j  0.8460 -1.0609e+00j  0.0000 -1.1102e-16j]


In [21]:
#print(np.dot(p_estimator, poly_a))
# if we let a = -a
#print
print(np.dot(p_estimator, fake_poly_a))

[ -1.3323e-15 -1.4433e-15j  -1.1657e-15 -1.2212e-15j   8.4601e-01 -1.0609e+00j   3.0489e+00 -1.5543e-15j   2.3705e+00 +2.9725e+00j  -6.7845e-01 +2.9725e+00j  -1.2225e+00 +5.8874e-01j]


## Full E matrix calculated

In [26]:
# now let's find the whole E and also epsilon
E_1=obtain_E(alpha, E_2, s)
# concatenate E_1 and E_2 to obtain E
E = np.concatenate((E_1, E_2), axis=0)

epsilon = obtain_epsilon(E)

#print(epsilon)

Obtaining E[0]
[[-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]
 [-29.3859+36.8488j -61.3621+29.5505j]]

Calculated Results:
[[-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]
 [-75.5929 +8.6430e-15j]]

Obtaining E[1]
[[-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j -75.5929 +8.6430e-15j]
 [-61.3621 +2.9550e+01j 

## finally let's look at the epsilon matrix we get

In [27]:
print(epsilon)

[[ -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j
   -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j  -3.7796e+01 -7.5910e-14j]
 [  4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j
    4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j   4.4663e-14 -7.8539e-14j]
 [  3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j
    3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j   3.3116e-14 +4.4695e-14j]
 [ -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j  -5.1483e-14 +2.0802e-15j
   -5.1483e-14 +2.080

## we if we give it a factor? say sqrt(n)

In [28]:
print(np.sqrt(n)*epsilon)

[[ -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j
   -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j]
 [  1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j
    1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j]
 [  8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j
    8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j]
 [ -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j  -1.3621e-13 +5.5038e-15j
   -1.3621e-13 +5.503

In [29]:
# let's try to fix it
tmp = np.sqrt(n)*epsilon
a = np.roll(tmp, s-1, axis=0)
print(a)

[[ -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j
   -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j  -1.0000e+02 +2.4065e-13j]
 [ -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j
   -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j  -1.0000e+02 -2.0084e-13j]
 [  1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j
    1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j   1.1817e-13 -2.0779e-13j]
 [  8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j   8.7618e-14 +1.1825e-13j
    8.7618e-14 +1.182